# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [81]:
# import libraries
import re
import pandas as pd
import numpy as np
import sqlite3
import pickle
from sqlalchemy import create_engine


# download necessary NLTK data
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/widyapuspitaloka/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/widyapuspitaloka/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/widyapuspitaloka/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/widyapuspitaloka/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [43]:
# load data from database
engine = create_engine('sqlite:///DisasterMessages.db')
df = pd.read_sql("SELECT * FROM messages", engine)
df.head()

,id,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
df.isnull().sum()

id                        0
message                   0
genre                     0
related                   0
request                   0
offer                     0
aid_related               0
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake                0
cold                

In [45]:
X = df['message']
Y = df[df.columns[3:]]

print(X.shape)

(26216,)


In [32]:
X

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
                               ...                        
26211    The training demonstrated how to enhance micro...
26212    A suitable candidate has been selected and OCH...
26213    Proshika, operating in Cox's Bazar municipalit...
26214    Some 2,000 women protesting against the conduc...
26215    A radical shift in thinking came about as a re...
Name: message, Length: 26216, dtype: object

In [22]:
labels = list(df.columns[3:])

### 2. Write a tokenization function to process your text data

In [46]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    '''
    input: text (str) - text to be processed
    output: clean token(list) -list of clean token
    '''
    #print('original_text', text)
    #remove stop words
    stop_words = stopwords.words("english")
    
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')

    #text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # extract word tokens from the text - tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        if tok not in stop_words:
            # lemmatize, normalize case, and remove leading/trailing white space
            clean_tok = lemmatizer.lemmatize(tok).lower().strip()
            clean_tokens.append(clean_tok)

    return clean_tokens

In [47]:
# Create a custom transformer to extract verb
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    '''
    A class verb extractor - an estimator to extract the starting verb in a sentence
    '''
    def starting_verb(self, text): 
        """
        Function to evaluate whether there is a verb starting a sentence
        in the text.
        INPUT:
        text (str) - text to search the verb
        """
        # tokenize by sentences
        sentence_list = nltk.sent_tokenize(text)
        
        for sentence in sentence_list:
            # tokenize each sentence into words and tag part of speech
            #print(pos_tags)
            pos_tags = nltk.pos_tag(tokenize(sentence))
            if len(pos_tags)>0:
            # index pos_tags to get the first word and part of speech tag
                first_word, first_tag = pos_tags[0]
        
        # return true if the first word is an appropriate verb or RT for retweet
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return True
        return False
        
    def fit(self, x, y=None):
        """
        Fit function is required for an estimator.
        """
        return self

    def transform(self, X):
        """
        Transfrom function to transform the data 
        Running self.starting_verb functin and return the data as X
        """
        # apply starting_verb function to all values in X
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)
            

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [74]:
def model_pipeline():
    '''
    Return a model that has been pipelined
    '''
    #build pipeline
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
    #specifity the parameter for grid search
    parameters = {
    'features__text_pipeline__tfidf__use_idf':(True, False),
    'clf__estimator__n_estimators': [10, 50], #50, 100
    'clf__estimator__min_samples_split': [2, 4], #4
    #'clf__estimator__learning_rate': [0.1, 0.3]
        }

    # create grid search object
    model = GridSearchCV(pipeline, parameters, n_jobs=1, verbose=1)

    return model


In [10]:
#checking parameters
#pipeline = model_pipeline()
#pipeline.get_params()

### 4. Train Pipeline
- Split data into train and test
- Train the pipeline

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline_model = model_pipeline()
pipeline_model.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x7fc978db5c10>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())])),
                                                                       ('starting_verb',
                                                                        StartingVerbExtractor())])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={})

In [50]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((19662,), (19662, 35), (6554,), (6554, 35))

In [ ]:
X_train = np.arange(707832).reshape((19662, 36))
X_train

In [ ]:
X_test = np.arange(235944).reshape((6554, 36))
X_test.shape

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [51]:
y_pred_test = pipeline_model.predict(X_test)
y_pred_train = pipeline_model.predict(X_train)

In [40]:
#classification report on the test data 
print(classification_report(y_test.values, y_pred_test, target_names=Y.columns.values))
print('--------------------------------------------------------------------')
#classification report on the train data 
print(classification_report(y_train.values, y_pred_train, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.95      0.88      4962
               request       0.80      0.48      0.60      1122
                 offer       0.00      0.00      0.00        21
           aid_related       0.73      0.69      0.71      2710
          medical_help       0.54      0.06      0.10       523
      medical_products       0.70      0.06      0.11       360
     search_and_rescue       0.83      0.03      0.05       186
              security       0.00      0.00      0.00       112
              military       0.70      0.13      0.22       204
                 water       0.88      0.30      0.45       405
                  food       0.81      0.62      0.71       738
               shelter       0.84      0.30      0.45       584
              clothing       0.62      0.10      0.17       100
                 money       1.00      0.01      0.03       138
        missing_people       0.00      

/Users/widyapuspitaloka/opt/miniconda3/envs/dsnd_etl/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/widyapuspitaloka/opt/miniconda3/envs/dsnd_etl/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/widyapuspitaloka/opt/miniconda3/envs/dsnd_etl/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _w

### 6. Improve your model
Use grid search to find better parameters. 

In [75]:
pipeline_model = model_pipeline()
pipeline_model.get_params()

{'cv': None,
 'error_score': nan,
 'estimator__memory': None,
 'estimator__steps': [('features',
   FeatureUnion(transformer_list=[('text_pipeline',
                                   Pipeline(steps=[('vect',
                                                    CountVectorizer(tokenizer=<function tokenize at 0x7fc978db5c10>)),
                                                   ('tfidf',
                                                    TfidfTransformer())])),
                                  ('starting_verb', StartingVerbExtractor())])),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'estimator__verbose': False,
 'estimator__features': FeatureUnion(transformer_list=[('text_pipeline',
                                 Pipeline(steps=[('vect',
                                                  CountVectorizer(tokenizer=<function tokenize at 0x7fc978db5c10>)),
                                                 ('tfidf',
                                              

In [76]:
# improving model with cvgrids (add parameters to the function)
pipeline_model.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x7fc978db5c10>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())])),
                                                                       ('starting_verb',
                                                                        StartingVerbExtractor())])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=1,
             param_grid={'clf__es

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [77]:
y_pred_test = pipeline_model.predict(X_test)
y_pred_train = pipeline_model.predict(X_train)

In [78]:
#classification report on the test data 
print(classification_report(y_test.values, y_pred_test, target_names=Y.columns.values))
print('--------------------------------------------------------------------')
#classification report on the train data 
print(classification_report(y_train.values, y_pred_train, target_names=Y.columns.values))

print('--------------------------------------------------------------------')
print('Best Parameters:', pipeline_model.best_params_)

                        precision    recall  f1-score   support

               related       0.83      0.96      0.89      4976
               request       0.85      0.47      0.61      1110
                 offer       0.00      0.00      0.00        28
           aid_related       0.75      0.67      0.71      2677
          medical_help       0.60      0.07      0.13       514
      medical_products       0.61      0.06      0.11       327
     search_and_rescue       0.69      0.14      0.24       166
              security       0.00      0.00      0.00       105
              military       0.67      0.06      0.10       181
                 water       0.86      0.33      0.47       405
                  food       0.82      0.56      0.67       699
               shelter       0.80      0.33      0.47       559
              clothing       0.50      0.04      0.08        94
                 money       0.80      0.03      0.05       159
        missing_people       1.00      

/Users/widyapuspitaloka/opt/miniconda3/envs/dsnd_etl/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/widyapuspitaloka/opt/miniconda3/envs/dsnd_etl/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/widyapuspitaloka/opt/miniconda3/envs/dsnd_etl/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _w

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [79]:
print('Best Parameters:', pipeline_model.best_params_)

Best Parameters: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 50, 'features__text_pipeline__tfidf__use_idf': True}


### 9. Export your model as a pickle file

In [82]:
file = open('DisasterMessages.pkl', 'wb')
pickle.dump(pipeline_model, file)
file.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.